#  Graph Theory and Power Flow

Energy Systems 

TU Berlin
***

**Imports**

In [1]:
import numpy as np
import numpy.linalg
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('bmh')
%matplotlib inline

***
## Exercise 1: Undirected Graph

**(a) Compile the nodes list and the edge list.**
> **Remark:** While graph-theoretically both lists are unordered sets, let's agree on an ordering now which can serve as basis for the matrices in the following exercises: we sort everything in ascending numerical order, i.e. node 1 before node 2 and edge (1,2) before (1,4) before (2,3).

In [2]:
nodes = [0, 1, 2, 3, 4, 5]
edges = [(0, 1), (1, 2), (1, 4),
         (1, 5), (2, 3), (2, 4),
         (3, 4), (4, 5)]

***
**(b) Determine the order and the size of the network.**

In [3]:
N = len(nodes)
E = len(edges)

In [4]:
print("Order: {}\nSize: {}".format(N, E))

Order: 6
Size: 8


***
**(c) Compute the adjacency matrix $A$ and check that it is symmetric.**
> In graph theory and computer science, an adjacency matrix is a square matrix used to represent a finite graph. The elements of the matrix indicate whether pairs of vertices are adjacent or not in the graph.

Adjacency Matrix:

In [5]:
A = np.zeros((N, N))

for u, v in edges:
    A[u, v] += 1
    A[v, u] += 1

A

array([[0., 1., 0., 0., 0., 0.],
       [1., 0., 1., 0., 1., 1.],
       [0., 1., 0., 1., 1., 0.],
       [0., 0., 1., 0., 1., 0.],
       [0., 1., 1., 1., 0., 1.],
       [0., 1., 0., 0., 1., 0.]])

Check for symmetry:

In [6]:
(A == A.T).all()

True

***
**(d) Find the degree $k_n$ of each node $n$ and compute the average degree of the network.**
> In graph theory, the degree (or valency) of a vertex of a graph is the number of edges incident to the vertex, with loops counted twice.

In [7]:
k = A.sum(axis=1)
k

array([1., 4., 3., 2., 4., 2.])

In [8]:
k.mean()

2.6666666666666665

***
**(e) Determine the incidence matrix $K$ by assuming the links are always directed from smaller-numbered node to larger-numbered node, i.e. from node 2 to node 3, instead of from 3 to 2.**
> The unoriented incidence matrix (or simply incidence matrix) of an undirected graph is a $n \times m$ matrix $B$, where n and m are the numbers of vertices and edges respectively, such that $B_{i,j} = 1$ if the vertex $v_i$ and edge $e_j$ are incident and 0 otherwise.

In [9]:
K = np.zeros((N,E))

for i, (u, v) in enumerate(edges):
    K[u,i] = 1
    K[v,i] = -1
    
K

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  1.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  1.,  0.],
       [ 0.,  0., -1.,  0.,  0., -1., -1.,  1.],
       [ 0.,  0.,  0., -1.,  0.,  0.,  0., -1.]])

***
**(f) Compute the Laplacian $L$ of the network using $k_n$ and $A$. Remember that the Laplacian can also be computed as $L=KK^T$ and check that the two definitions agree.**
> The **Laplacian** (also: admittance matrix, Kirchhoff matrix, discrete Laplacian) is a matrix representation of a graph. It is defined as the difference of degree matrix and adjacency matrix. The **degree matrix** is a diagonal matrix which contains information about the degree of each vertex.

In [10]:
D = np.diag(A.sum(axis=1))

In [11]:
L = D - A
L

array([[ 1., -1.,  0.,  0.,  0.,  0.],
       [-1.,  4., -1.,  0., -1., -1.],
       [ 0., -1.,  3., -1., -1.,  0.],
       [ 0.,  0., -1.,  2., -1.,  0.],
       [ 0., -1., -1., -1.,  4., -1.],
       [ 0., -1.,  0.,  0., -1.,  2.]])

In [12]:
np.array_equal(L, K.dot(K.T))

True

***
## Execise 3: Jupyter Notebook Exercise

If you map the nodes to `1=DK, 2=DE, 3=CH, 4=IT, 5=AT, 6=CZ` the network represents a small part of the European electricity network (albeit very simplified). In the repository, you can find the power imbalance time series for the six countries for January 2017 in hourly MW at `./data/imbalance.csv`. They have been derived from physical flows as published by [ENTSO-E](https://transparency.entsoe.eu/transmission-domain/physicalFlow/show)

The linear power flow is given by
$$p_i = \sum_j \tilde{L}_{i,j}\theta_j \qquad \text{and} \qquad f_l = \frac{1}{x_l} \sum_i K_{i,l}\theta_i, \qquad \text{where} \qquad \tilde{L}_{i,j}= \sum_l K_{i,l}\frac{1}{x_l} K_{j,l}$$
is the weighted Laplacian. For simplicity, we assume identity reactance on all links $x_l = 1$.

***
**Read data**

In [13]:
imbalance = pd.read_csv('data/imbalance.csv', index_col=0, parse_dates=True)
imbalance.head()

DK       DE       CH     IT       AT       CZ
DateTime                                                              
2017-01-01 00:00:00  -79.73  3486.25 -3403.37  -97.5 -2867.33  2961.68
2017-01-01 01:00:00 -109.14  4248.17 -3829.59 -105.9 -3149.98  2946.44
2017-01-01 02:00:00 -101.49  5314.56 -4131.66 -199.2 -3646.87  2764.66
2017-01-01 03:00:00 -197.96  5450.71 -4009.61 -656.0 -3345.56  2758.42
2017-01-01 04:00:00 -526.15  5695.72 -3924.43 -646.2 -3393.12  2794.18

\begin{equation}
p_u = \sum_v L_{u,v} \theta_v
\end{equation}

***
**(a) Compute the voltage angles $\theta_j$ and flows $f_l$ for the first hour in the dataset with the convention of $\theta_0 = 0$; i.e. the slack bus is at node 0.**
> **Remark:** Linear equation systems are solved efficiently using [`numpy.linalg.solve`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.solve.html).

Calculate the *voltage angles* first.
> Note, that we define the node for Denmark as slack and therefore force $\theta_{DK}=0$.

In [14]:
imbalance.iloc[0].values[1:]

array([ 3486.25, -3403.37,   -97.5 , -2867.33,  2961.68])

In [15]:
theta = np.r_[0, np.linalg.solve(L[1:,1:], imbalance.iloc[0].values[1:])]
theta

array([    0.        ,    79.73      , -2302.97857143, -1995.25809524,
       -1590.03761905,   725.68619048])

Then, calculate the *flows*:

In [16]:
flows = K.T.dot(theta)
flows

array([  -79.73      ,  2382.70857143,  1669.76761905,  -645.95619048,
        -307.72047619,  -712.94095238,  -405.22047619, -2315.72380952])

***
**(b) Determine the average flow on each link for January 2017 and draw it as a directed network**

> **Hint:** You may want to make use of the function `np.vstack`.

In [17]:
flows = K.T.dot(np.vstack([np.zeros((1, len(imbalance))), np.linalg.solve(L[1:,1:], imbalance.values[:,1:].T)]))

In [18]:
flows.shape

(8, 744)

In [19]:
avg_flows = flows.mean(axis=1)
avg_flows

array([  451.02294355,  2402.42223502,  2124.94871608,   426.46578277,
         421.1171115 ,  -277.47351895,  -698.59063044, -1698.48293331])